In [10]:
#coding=utf-8
import os
import os.path
import sys
from osgeo import gdal
#from gdalconst import *
from osgeo import gdalconst
#import osr
from osgeo import ogr
import numpy as np
#import mk
import pandas as pd
#from statsmodels.formula.api import ols
#from statsmodels.stats.anova import anova_lm

In [13]:
def Read(RasterFile):#读取每个图像的信息
    ds = gdal.Open(RasterFile,GA_ReadOnly)    
    if ds is None:
        #print 'Cannot open ',RasterFile
        print(f"Can not open {RasterFile}")
        sys.exit(1)
    cols = ds.RasterXSize
    rows = ds.RasterYSize
    band = ds.GetRasterBand(1)
    data = band.ReadAsArray(0,0,cols,rows)  
    return data

def Readxy(RasterFile): #读取每个图像的坐标信息并返回     
    ds = gdal.Open(RasterFile,GA_ReadOnly)
    if ds is None:
        #print 'Cannot open ',RasterFile
        print(f"Can not open {RasterFile}")
        sys.exit(1)
    cols = ds.RasterXSize
    rows = ds.RasterYSize
    band = ds.GetRasterBand(1)
    #data = band.ReadAsArray(0,0,cols,rows)M
    noDataValue = band.GetNoDataValue()
    projection=ds.GetProjection()
    geotransform = ds.GetGeoTransform()
    return rows,cols,geotransform,projection,noDataValue

def WriteGTiffFile(filename, nRows, nCols, data,geotrans,proj, noDataValue, gdalType):#向磁盘写入结果文件
    format = "GTiff"   
    driver = gdal.GetDriverByName(format)
    ds = driver.Create(filename, nCols, nRows, 1, gdalType)
    ds.SetGeoTransform(geotrans)
    ds.SetProjection(proj)
    ds.GetRasterBand(1).SetNoDataValue(noDataValue)
    ds.GetRasterBand(1).WriteArray(data)    
    ds = None
    

In [ ]:
rows,cols,geotransform,projection,noDataValue = Readxy("D:/china/2000.tif") 
print rows,cols
count=0
rootdir = "D:/china"
datalist=[]
for dirpath,filename,filenames in os.walk(rootdir):#遍历源文件
    for filename in filenames:
        if os.path.splitext(filename)[1] == '.tif':#判断是否为tif格式
             filepath = os.path.join(dirpath,filename)
             filedata1 = [[0.0]*cols]*rows
             
             filedata2 = np.array(filedata1)
             filedata3 = Read(filepath)
             count+=1
             datalist.append(filedata3)
             
dtarelist=[]
for m in range(len(datalist)):
    dtarelist.append((datalist[m].reshape((1,396, 697))))
#print dtarelist
#print dtarelist[2].shape
for a in range(len(datalist)):
  if a==0:
    dtaz=np.concatenate([dtarelist[a],dtarelist[a+1]],axis=0)
  if a>1:
    dtaz=np.concatenate([dtaz,dtarelist[a]],axis=0)
#print dtaz.shape

    
Arraytime=np.array([0,1,2,3,4,5,6,7,8,9,10,11,12,13,14,15])

Time=np.vstack([Arraytime,np.ones(len(Arraytime))]).T
temparray=[]
XL=np.zeros((396, 697),dtype=np.float)
XF=np.zeros((396, 697),dtype=np.float)
for r in range(396):
     for c in range(697):
         for k in range(16):
              temparray.append(dtaz[k,r,c])
         data={'Time':Time[:,0],'Arrayslope':temparray}
         df=pd.DataFrame(data)
         #Arrayslope=np.array(temparray)
         temparray=[]
         formula='Arrayslope~Time'
         model=ols(formula,df).fit()
         f=anova_lm(model)
         m=model.params.Time
         #m,b=np.linalg.lstsq(Time,Arrayslope)[0]
         XF[r,c]=f.iloc[0,3]
         XL[r,c]=m
     print r
        
WriteGTiffFile("D:/cal/NDVIMEAN_L.tif",rows,cols,XL,geotransform,projection,noDataValue, GDT_Float32)
WriteGTiffFile("D:/cal/NDVIMEAN_F.tif",rows,cols,XF,geotransform,projection,noDataValue, GDT_Float32)                